## DiNMF

In [30]:
import torch
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

from util import eval, loadDataset, normalization

eps = torch.tensor(10 ** -10)

# data = torch.load('../Datasets/3Sources.npy')
# V = len(data)-1
# Y = data[-1].flatten()
# c = len(np.unique(Y))

# X0 = data[:-1]

datasets = ['3Sources.npy',
 'BBCSport.npy',
 'Caltech101.npy',
 'Caltech_2.npy',
 'Citeseer.npy',
 'Coil100.npy',
 'Cora.npy',
 'EYaleB10.npy',
 'Handwritten.npy',
 'MNIST10.npy',
 'UCIdigit.npy',
 'Umist.npy',
 'Yale32.npy',
 'Yeast.npy']


X0, Y, V, c = loadDataset(8)

r = 20
alpha = 1
beta = 1
iter = 300

W = []
H = []
X = []

for v in range(V):
    
    X.append(torch.tensor(X0[v]).type(torch.float32))

    # Normalization
    # X[v] = normalization(X[v], 'L2col')
    
    d, n = X[v].shape

    W.append(torch.rand(d, r))
    H.append(torch.rand(r, n))
    

    
# Optimization
err = torch.zeros(iter)

for t in range(iter):

    for v in range(V):

        
        
        # Updating Ws
        Wn = X[v]        @ H[v].T 
        Wd = W[v] @ H[v] @ H[v].T 
        W[v] = W[v] * (Wn / torch.maximum(Wd, eps))


        # Updating Hs
        Q = torch.sum(torch.stack(H), dim=0)
        
        Hn = 2 * (W[v].T @ X[v])                    
        Hd = 2 * (W[v].T @ W[v] @ H[v]) + alpha * (Q - H[v]) + 2 * beta * H[v] 
        H[v] = H[v] * (Hn / torch.maximum(Hd, eps))

        # Calculating cost function
        # err[t] += torch.norm(X[v] - W[v] @ H[v]) ** 2 + alpha * (torch.trace(H[v] @ (Q - H[v]).T)) + beta * torch.norm(H[v])**2
           
# plt.plot(err)

Hfinal = torch.mean(torch.stack(H), dim=0)

pred = KMeans(n_clusters=c, n_init='auto').fit(Hfinal.T).labels_

nmi, acc, ari, f1mi, f1ma, f1we, pur = eval(Y, pred)
nmi, acc, ari, f1mi, f1ma, f1we, pur

C:\Users\amjad\AppData\Local\Temp\ipykernel_10488\4191043127.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X.append(torch.tensor(X0[v]).type(torch.float32))


(0.6289322710724004,
 0.659,
 0.5035182647389292,
 0.659,
 0.6549953287547585,
 0.6549953287547586,
 0.659)

## LP-DiNMF

In [74]:
import torch
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph

from util import eval, loadDataset, normalization

eps = torch.tensor(10 ** -10)

datasets = ['3Sources.npy',
 'BBCSport.npy',
 'Caltech101.npy',
 'Caltech_2.npy',
 'Citeseer.npy',
 'Coil100.npy',
 'Cora.npy',
 'EYaleB10.npy',
 'Handwritten.npy',
 'MNIST10.npy',
 'UCIdigit.npy',
 'Umist.npy',
 'Yale32.npy',
 'Yeast.npy']


X0, Y, V, c = loadDataset(8)

r = 100
k = 6
alpha = .1
beta = .1
gamma = 0.001
iter = 500

W = []
H = []
X = []

for v in range(V):
    
    X.append(torch.tensor(X0[v]).type(torch.float32))

    # Normalization
    # X[v] = normalization(X[v], 'L1row')
    
    d, n = X[v].shape
    
    W.append(torch.rand(d, r))
    H.append(torch.rand(r, n))
    
A = []
D = []
for v in range(V):
    
    A0 = torch.tensor(kneighbors_graph(X[v].T, k, mode='connectivity', include_self=False).toarray()).type(torch.float32)
    A.append(torch.maximum(A0, A0.T))
    D.append(torch.diag(torch.sum(A0, dim = 1)))

    
# Optimization
err = torch.zeros(iter)

for t in range(iter):

    for v in range(V):

        # Updating Hs
        Q = torch.sum(torch.stack(H)) + H[v]
        
        Hn = 2 * (W[v].T @ X[v])                    + 2 * gamma * (H[v] @ A[v])
        Hd = 2 * (W[v].T @ W[v] @ H[v]) + alpha * Q + 2 * gamma * (H[v] @ D[v])
        H[v] = H[v] * (Hn / torch.maximum(Hd, eps))
        
        # Updating Ws
        Wn = X[v]        @ H[v].T 
        Wd = W[v] @ H[v] @ H[v].T 
        W[v] = W[v] * (Wn / torch.maximum(Wd, eps))
        
        

        # Calculating cost function
#         err[t] += torch.norm(X[v] - W[v] @ H[v]) ** 2 + gamma * torch.trace(H[v] @ (D[v] - A[v]) @ H[v].T) 
           
# plt.plot(err)

Hfinal = torch.mean(torch.stack(H), dim=0)

pred = KMeans(n_clusters=c, n_init='auto').fit(Hfinal.T).labels_

nmi, acc, ari, f1mi, f1ma, f1we, pur = eval(Y, pred)
nmi, acc, ari, f1mi, f1ma, f1we, pur

C:\Users\amjad\AppData\Local\Temp\ipykernel_10488\1829871985.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X.append(torch.tensor(X0[v]).type(torch.float32))


(0.7531856071383977,
 0.782,
 0.6578366645037367,
 0.782,
 0.7477166686377335,
 0.7477166686377335,
 0.782)